# PART 1 : SEGMENTATION + NOISE ADD

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import data_augmentation
import NN_segmentation
import numpy as np

ModuleNotFoundError: No module named 'UNET3d_D4'

In [13]:
path_data = r"/home/jovyan/project_data/ACDC-Daten"
add_noise = True

data_list = []
label_list = []

for root, _, files in os.walk(path_data):
    if os.path.isfile(os.path.join(root, "image.nii.gz")):
        data_list.append(NN_segmentation.tst_dataset.cardiacdata(os.path.join(path_data, "image.nii.gz")), os.path.join(path_data, "label.nii.gz"))

AttributeError: module 'NN_segmentation' has no attribute 'tst_dataset'

In [12]:
data_list[0]

IndexError: list index out of range